# Pharmacovigilance Analysis using FAERS database and deduplication according to VigiMatch

Version: 1  
Author: Djamilla Simoens

Please make sure your read the readme and work using a Python venv.

In [1]:
# =============================
# 2) Import Following Libraries
# =============================

import pandas as pd
import numpy as np
import os
import glob
from scipy.stats import fisher_exact, chi2_contingency
from tqdm import tqdm


In [14]:
# =============================
# 3) User Input
# =============================

# Example input (replace by your own values)
# Provide multiple names for drugs and adverse events

drug_synonyms = ['capivasertib', 'TRUQAP']            # fill in quotes the drug of interest 
event_synonyms = ['Stomatitis']                       # fill in quots all adverse reactions of interest
data_dir = './data-source'                            # Update with your folder with the FAERS data, make sure you have .txt files
output_file = 'FAERS_capivasertib_term3_Results.xlsx' # File name to write an Excel file to
csv_log_output= "csv_log_output.csv"                  # File to write error logs to

start_year_q = (2023, 4)                   # Year and quarter start
end_year_q = (2025, 1)                     # Year and quarter end

skipped_rows_log = []

print("Cell complete")

Cell complete


In [ ]:
# =============================
# Misc fuctions
# =============================
def is_in_range(fname, start, end):
    import re
    match = re.search(r'(\d{2})(Q[1-4])', fname.upper())
    if not match:
        return False
    year_str, quarter_str = match.groups()
    year = int('20' + year_str) if int(year_str) < 50 else int('19' + year_str)
    quarter = int(quarter_str[1])
    return (year, quarter) >= start and (year, quarter) <= end

# =============================
# 4) Load FAERS Data
# =============================

def load_files(folder, pattern, start_range, end_range):
    pattern_lower = pattern
    pattern_upper = pattern.upper()
    files = sorted(glob.glob(os.path.join(folder, pattern_lower)) +
                   glob.glob(os.path.join(folder, pattern_upper)))

    filtered_files = [f for f in files if is_in_range(os.path.basename(f), start_range, end_range)]

    dfs = []
    for file in tqdm(filtered_files, desc=f"Loading {pattern.split('*')[0]}"):
        try:
            df = pd.read_csv(
                file,
                sep='$',
                dtype=str,
                encoding='utf-8',
                on_bad_lines='skip',
                low_memory=False
            )
            dfs.append(df)
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(
                    file,
                    sep='$',
                    dtype=str,
                    encoding='latin1',
                    on_bad_lines='skip',
                    low_memory=False
                )
                dfs.append(df)
            except Exception as e:
                print(f"Failed to load {file}: {e}")
                skipped_rows_log.append((file, str(e)))
        except Exception as e:
            print(f"Failed to load {file}: {e}")
            skipped_rows_log.append((file, str(e)))

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

df_drug = load_files(data_dir, 'DRUG*.txt', start_year_q, end_year_q)
df_reac = load_files(data_dir, 'REAC*.txt', start_year_q, end_year_q)

print(f"Total DRUG records: {len(df_drug)}")
print(f"Total REACTION records: {len(df_reac)}")
all_isr = set(df_drug['primaryid'].unique())


Loading REAC: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]

Total DRUG records: 2008162
Total REACTION records: 1432926


In [ ]:
# =============================
# 5) Deduplication According to VigiMatch Standards
# =============================

print("Filtering suspect drugs...")
df_drug_ps = df_drug[df_drug['role_cod'].str.upper() == 'PS'].copy()
df_drug_ps['drugname'] = df_drug_ps['drugname'].fillna('').str.upper()
df_reac['pt'] = df_reac['pt'].fillna('').str.upper()

def normalize_name(name):
    if not isinstance(name, str):
        return ''
    name = name.upper()
    for synonym in drug_synonyms:
        if synonym.upper() in name:
            return synonym.upper()
    return name

df_drug_ps['drugname_norm'] = df_drug_ps['drugname'].apply(normalize_name)

dedup_columns = ['primaryid', 'drugname_norm', 'route', 'dose_vbm']
for col in dedup_columns:
    if col not in df_drug_ps.columns:
        df_drug_ps[col] = ''

df_drug_ps_before = len(df_drug_ps)
df_drug_ps_unique = df_drug_ps.sort_values(by='primaryid').drop_duplicates(subset=dedup_columns)
df_drug_ps_after = len(df_drug_ps_unique)
duplicates_removed_drug = df_drug_ps_before - df_drug_ps_after

print(f"VigiMatch-style DRUG duplicates removed: {duplicates_removed_drug} (from {df_drug_ps_before} → {df_drug_ps_after})")

Filtering suspect drugs...
VigiMatch-style DRUG duplicates removed: 13929 (from 431702 → 417773)


In [ ]:
# =============================
# 6) Matching Drug and Event (and synonym breakdown)
# =============================

# Build set of primaryids for drug synonyms
isr_drug = set()
for name in drug_synonyms:
    name = name.upper()
    matches = df_drug_ps_unique[df_drug_ps_unique['drugname_norm'] == name]['primaryid'].unique()
    isr_drug.update(matches)
    
# Build set of primaryids for AE synonyms
isr_event = set()
for name in event_synonyms:
    name = name.upper()
    matches = df_reac[df_reac['pt'].str.contains(name, na=False, regex=False)]['primaryid'].unique()
    isr_event.update(matches)


# Event synonym breakdown - list all AE of interests and their specific count
breakdown_list = []
for event_term in event_synonyms:
    event_term_upper = event_term.upper()
    isr_event_term = set(df_reac[df_reac['pt'].str.contains(event_term_upper, na=False, regex=False)]['primaryid'].unique())
    
    A_term = len(isr_drug & isr_event_term)  # Drug + Event
    B_term = len(isr_drug - isr_event_term)  # Drug + No Event
    C_term = len(isr_event_term - isr_drug)  # Event + No Drug
    D_term = len(all_isr - (isr_drug | isr_event_term))  # Neither
    
    breakdown_list.append({
        'Event Synonym': event_term,
        'A (Drug+Event)': A_term,
        'B (Drug+No Event)': B_term,
        'C (Event+No Drug)': C_term,
        'D (Neither)': D_term
    })

breakdown_df = pd.DataFrame(breakdown_list)

print("Breakdown of counts per event synonym:")
print(breakdown_df)


Breakdown of counts per event synonym:
  Event Synonym  A (Drug+Event)  B (Drug+No Event)  C (Event+No Drug)  \
0    Stomatitis               4                245               1696   

   D (Neither)  
0       398569  


In [7]:
# =============================
# 7) Build 2x2 Contingency Table
# =============================

A = len(isr_drug & isr_event)  # Drug + Event
B = len(isr_drug - isr_event)  # Drug + No Event
C = len(isr_event - isr_drug)  # Event + No Drug
D = len(all_isr - (isr_drug | isr_event))  # Neither

print(f"Contingency Table:\nA (Drug+Event): {A}\nB (Drug+No Event): {B}\nC (Event+No Drug): {C}\nD (Neither): {D}")


Contingency Table:
A (Drug+Event): 4
B (Drug+No Event): 245
C (Event+No Drug): 1696
D (Neither): 398569


In [9]:
# =============================
# 8) Statistical Analysis
# =============================
contingency_table = np.array([[A, B], [C, D]])

# ROR calculation
ROR = (A / B) / (C / D) if B > 0 and C > 0 and D > 0 else np.nan

# PRR calculation
prr_numerator = A / (A + B) if (A + B) > 0 else np.nan
prr_denominator = (A + C) / (A + B + C + D) if (A + B + C + D) > 0 else np.nan
PRR = prr_numerator / prr_denominator if prr_denominator > 0 else np.nan

# Fisher's exact test
oddsr, p_fisher = fisher_exact(contingency_table)

# Chi-square test without continuity correction
chi2, p_chi2, dof, expected = chi2_contingency(contingency_table, correction=False)

# Chi-square test with Yates continuity correction
chi2_yates, p_chi2_yates, dof_yates, expected_yates = chi2_contingency(contingency_table, correction=True)

# Standard Error and 95% CI for ROR
SE = np.sqrt(1/A + 1/B + 1/C + 1/D) if A*B*C*D > 0 else np.nan
lower_CI = np.exp(np.log(ROR) - 1.96 * SE) if not np.isnan(SE) else np.nan
upper_CI = np.exp(np.log(ROR) + 1.96 * SE) if not np.isnan(SE) else np.nan

# Standard Error and 95% CI for PRR
if A > 0 and B > 0 and C > 0 and D > 0 and PRR > 0:
    SE_log_PRR = np.sqrt(1/A - 1/(A+B) + 1/C - 1/(C+D))
    lower_CI_PRR = np.exp(np.log(PRR) - 1.96 * SE_log_PRR)
    upper_CI_PRR = np.exp(np.log(PRR) + 1.96 * SE_log_PRR)
else:
    lower_CI_PRR = np.nan
    upper_CI_PRR = np.nan

# Relative Reporting Ratio (RRR) and 95% CI
if (A + B) > 0 and (C + D) > 0 and A > 0 and C > 0:
    RRR = (A / (A + B)) / (C / (C + D))
    SE_log_RRR = np.sqrt(1/A - 1/(A+B) + 1/C - 1/(C+D))
    lower_CI_RRR = np.exp(np.log(RRR) - 1.96 * SE_log_RRR)
    upper_CI_RRR = np.exp(np.log(RRR) + 1.96 * SE_log_RRR)
else:
    RRR = np.nan
    lower_CI_RRR = np.nan
    upper_CI_RRR = np.nan

# Haldane's Odds Ratio (HOR) and 95% CI with 0.5 continuity correction
a_c = A + 0.5
b_c = B + 0.5
c_c = C + 0.5
d_c = D + 0.5
HOR = (a_c * d_c) / (b_c * c_c)
SE_log_HOR = np.sqrt(1/a_c + 1/b_c + 1/c_c + 1/d_c)
lower_CI_HOR = np.exp(np.log(HOR) - 1.96 * SE_log_HOR)
upper_CI_HOR = np.exp(np.log(HOR) + 1.96 * SE_log_HOR)

print("Cell calculation succesful")

Cell calculation succesful


In [12]:
# =============================
# 9. Results and Excel Data Output
# =============================
results = {
    'Database': 'FAERS',
    'Drug(s)': ", ".join(drug_synonyms),
    'Event(s)': ", ".join(event_synonyms),
    'Start Quarter': f"Q{start_year_q[1]} {start_year_q[0]}",
    'End Quarter': f"Q{end_year_q[1]} {end_year_q[0]}",
    'A (Drug+Event)': A,
    'B (Drug+No Event)': B,
    'C (Event+No Drug)': C,
    'D (Neither)': D,
    'ROR': ROR,
    'ROR 95% CI Lower': lower_CI,
    'ROR 95% CI Upper': upper_CI,
    'PRR': PRR,
    'PRR 95% CI Lower': lower_CI_PRR,
    'PRR 95% CI Upper': upper_CI_PRR,
    'RRR': RRR,
    'RRR 95% CI Lower': lower_CI_RRR,
    'RRR 95% CI Upper': upper_CI_RRR,
    'Haldane OR': HOR,
    'Haldane OR 95% CI Lower': lower_CI_HOR,
    'Haldane OR 95% CI Upper': upper_CI_HOR,
    'Fisher p-value': p_fisher,
    'Chi2 p-value': p_chi2,
    'Chi2 Yates': chi2_yates,
    'Chi2 Yates p-value': p_chi2_yates,
    'DRUG Duplicates Removed': duplicates_removed_drug,
    'Original DRUG Records': df_drug_ps_before,
    'Skipped Files': len(skipped_rows_log)
}

result_df = pd.DataFrame([results])
print(result_df)

print(f"ROR (Reporting Odds Ratio) = {ROR:.2f} with 95% CI [{lower_CI:.2f}, {upper_CI:.2f}]")
print(f"PRR (Proportional Reporting Ratio) = {PRR:.2f} with 95% CI [{lower_CI_PRR:.2f}, {upper_CI_PRR:.2f}]")
print(f"Relative Reporting Ratio (RRR) = {RRR:.2f} with 95% CI [{lower_CI_RRR:.2f}, {upper_CI_RRR:.2f}]")
print(f"Haldane's Odds Ratio (HOR) = {HOR:.2f} with 95% CI [{lower_CI_HOR:.2f}, {upper_CI_HOR:.2f}]")
print(f"Fisher's Exact test p-value = {p_fisher:.4g}")
print(f"Chi-square test (no correction) p-value = {p_chi2:.4g}")
print(f"Chi-square test (Yates correction) = {chi2_yates:.2f}, p-value = {p_chi2_yates:.4g}")

with pd.ExcelWriter(output_file) as writer:
    result_df.to_excel(writer, sheet_name='Summary', index=False)
    breakdown_df.to_excel(writer, sheet_name='Event_Synonym_Breakdown', index=False)

print(f"Results and breakdown exported to {output_file}")

if skipped_rows_log:
    pd.DataFrame(skipped_rows_log, columns=["File", "Error"]).to_csv(csv_log_output, index=False)
    print("Skipped file errors saved to 'skipped_files_log.csv'")


  Database               Drug(s)    Event(s) Start Quarter End Quarter  \
0    FAERS  capivasertib, TRUQAP  Stomatitis       Q4 2023     Q1 2025   

   A (Drug+Event)  B (Drug+No Event)  C (Event+No Drug)  D (Neither)  \
0               4                245               1696       398569   

        ROR  ...  Haldane OR  Haldane OR 95% CI Lower  \
0  3.836821  ...    4.306369                 1.692986   

   Haldane OR 95% CI Upper  Fisher p-value  Chi2 p-value  Chi2 Yates  \
0                10.953906        0.022353      0.004107    5.675083   

   Chi2 Yates p-value  DRUG Duplicates Removed  Original DRUG Records  \
0            0.017208                    13929                 431702   

   Skipped Files  
0              0  

[1 rows x 28 columns]
ROR (Reporting Odds Ratio) = 3.84 with 95% CI [1.43, 10.32]
PRR (Proportional Reporting Ratio) = 3.78 with 95% CI [1.43, 10.02]
Relative Reporting Ratio (RRR) = 3.79 with 95% CI [1.43, 10.03]
Haldane's Odds Ratio (HOR) = 4.31 with 95% CI 